<a href="https://colab.research.google.com/github/Solajungq/ta_indiv/blob/main/TA_Individual_Assignment_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Select Specific Domain
Research in Sustainable energy and AI

# 2. Data collection
Recent trend and analysis on renewable energy
Collect data from the recent market, business, technology analysis from kaggle data worldwide

# 3. Import data
Renewable Energy Worldwide Dec2023-Feb2024
Source: https://www.kaggle.com/datasets/vikashnewsdata1/renewable-energy?select=NewsData.io_Renewable_Energy.json

"This dataset contains data from a wide range of sources, including government reports, international organizations, and research institutions." (cited from the datasource in Kaggle)




In [ ]:
!pip install requests

In [ ]:
import requests

url2 = 'https://raw.githubusercontent.com/Solajungq/ta_indiv/main/NewsData.io_Renewable_Energy.json'
response = requests.get(url2)

with open('NewsData.io_Renewable_Energy.json', 'wb') as file:
    file.write(response.content)

In [ ]:
import pandas as pd

# Read the downloaded JSON file into a Pandas DataFrame
ww_df = pd.read_json('NewsData.io_Renewable_Energy.json')

# Display the DataFrame
ww_df

,article_id,title,link,keywords,creator,video_url,description,content,pubDate,image_url,...,source_url,source_icon,source_priority,country,category,language,ai_tag,ai_region,sentiment,sentiment_stats
0,34fa2a78349418b971cbce5686908804,Saudi sovereign wealth fund pitches kingdom as...,https://www.brecorder.com/news/40290372/saudi-...,markets,none@none.com (Reuters),NaN,MIAMI: The head of Saudi Arabia’s Public Inves...,MIAMI: The head of Saudi Arabia’s Public Inves...,2024-02-23 09:50:34,https://i.brecorder.com/large/2024/02/23145001...,...,https://www.brecorder.com,https://i.bytvi.com/domain_icons/brecorder.png,384715,pakistan,top,english,artificial intelligence,"miami,florida,united states of america,north a...",neutral,"positive:0.06,neutral:99.85,negative:0.09"
1,9a28fd4fd309b17db653ea86e4c7b8c1,Govt Amends Electricity Rules To Further Empow...,https://odishabytes.com/govt-amends-electricit...,india,OB Bureau,NaN,New Delhi: The Union government has amended th...,New Delhi: The Union government has amended th...,2024-02-23 09:12:16,https://odishabytes.com/wp-content/uploads/202...,...,https://odishabytes.com,https://i.bytvi.com/domain_icons/odishabytes.jpg,512881,india,top,english,"renewable energy,energy","new delhi,delhi,india,asia",neutral,"positive:0.84,neutral:99.03,negative:0.13"
2,81bdbec184bb3cfa549c1d75dd1ef4a9,CIP's journey through offshore wind turbulence,https://www.digitimes.com/news/a20240222VL204/...,sustainability,DIGITIMES,NaN,The offshore wind power industry is navigating...,The offshore wind power industry is navigating...,2024-02-23 08:42:05,https://img.digitimes.com/newsshow/20240222vl2...,...,https://www.digitimes.com,None,163335,china,top,english,"renewable energy,energy",None,neutral,"positive:3.73,neutral:95.96,negative:0.32"
3,e9c00bb7e42f0ee78d40221608feb878,BIAL ties up with CleanMax for 45.9 MW solar-w...,https://www.thehindu.com/business/bial-ties-up...,business,None,NaN,They have signed a 25-year long-term power pur...,Clean Max Enviro Energy Solutions (CleanMax) a...,2024-02-23 07:27:52,https://th-i.thgim.com/public/incoming/3e6mg1/...,...,https://www.thehindu.com,https://i.bytvi.com/domain_icons/thehindu.png,2972,india,top,english,"renewable energy,energy","bengaluru,karnataka,india,asia",positive,"positive:92.32,neutral:7.3,negative:0.38"
4,d0acebec6222d82a927acae52d3664a3,Union Power and New & Renewable Energy Ministe...,https://orissadiary.com/union-power-and-new-re...,"national,top news,ministry of power",Odisha Diary bureau,NaN,The Government of India has approved amendment...,The Government of India has approved amendment...,2024-02-23 07:22:55,https://orissadiary.com/wp-content/uploads/202...,...,https://orissadiary.com,None,2836164,india,top,english,"renewable energy,energy",None,positive,"positive:53.9,neutral:45.47,negative:0.63"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,a6b96628cf6333a70f01d1a0d7357a03,'Urban Mining' Offers Green Solution To Old So...,https://www.ibtimes.com/urban-mining-offers-gr...,None,None,NaN,As the world pivots from planet-warming fossil...,INTERNATIONAL BUSINESS TIMES NEWSLETTER SIGNUP...,2023-12-23 01:21:22,None,...,https://www.ibtimes.com,https://i.bytvi.com/domain_icons/ibtimes.png,458722,united states of america,top,english,None,None,None,None
3750,db483cf8d8c37bc4e338e42319175a37,'Urban mining' offers green solution to old so...,https://www.postguam.com/news/national/urban-m...,None,Paula RAMON,NaN,As the world pivots from planet-warming fossil...,As the world pivots from planet-warming fossil...,2023-12-23 01:18:47,https://bloximages.newyork1.vip.townnews.com/p...,...,https://www.postguam.com,None,93374,united states of america,top,english,None,None,None,None
3751,bed39a1c384cf2bae30106b458e1ed55,Trump win in 2024 could harm fight against cli...,https://www.arabnews.com/node/2430526/world,None,romulo,NaN,OTTAWA: If Republican frontrunner Donald Trump...,https://arab.news/c7ame OTTAWA: If Republican ...,

## Data cleaning
Clean data as we will need description written in English, no duplicated data



In [ ]:
import pandas as pd

# Filter the DataFrame to include only the rows where 'language' is 'English'
ww_df_c = ww_df[ww_df['language'] == 'english']

# Select only the columns 'description'
ww_df_c = ww_df_c[['description']]

# Drop duplicates
ww_df_c = ww_df_c.drop_duplicates()

# Drop rows with missing values in the 'description' column
ww_df_c = ww_df_c.dropna()

# Display the cleaned dataframe
ww_df_c



,description
0,MIAMI: The head of Saudi Arabia’s Public Inves...
1,New Delhi: The Union government has amended th...
2,The offshore wind power industry is navigating...
3,They have signed a 25-year long-term power pur...
4,The Government of India has approved amendment...
...,...
3746,The renewable energy company Drax welcomed 800...
3747,"article author: ReutersAuthor: Sat, 2023-12-23..."
3748,As the world pivots from planet-warming fossil...
3751,OTTAWA: If Republican frontrunner Donald Trump...


In [ ]:
# Filter the DataFrame to include only the rows where 'description' contains 'AI' or 'Artificial intelligence'
ww_df_c_ai = ww_df_c[ww_df_c['description'].str.contains('Artificial intelligence', case=False)]

# Display the filtered dataframe
print(ww_df_c_ai)


                                            description
0     MIAMI: The head of Saudi Arabia’s Public Inves...
7     In a recent development, Yasir Al-Rumayyan, th...
201   Power Optimizer Market Size, Share, Industry T...
210   Vietnam semiconductor market is projected to e...
216   TEXAS CITY, TEXAS, UNITED STATES, February 21,...
...                                                 ...
3512  ISTANBUL, TURKEY, December 28, 2023 /EINPressw...
3558  Digitalization is both a significant divider a...
3614  NEW YORK, NEW YORK, UNITED STATES, December 26...
3654  ARCHITECTURE and design, like all other sector...
3656  Several new technologies are lending themselve...

[64 rows x 1 columns]


In [ ]:
ww_df_c_ai.to_excel('filtered_data.xlsx', index=False)

# 4. Vector database

code reference to build vector database: https://github.com/RDGopal/IB9CW0-Text-Analytics/blob/main/day_seven_vector_dbs.ipynb

chunking strategy: https://github.com/RDGopal/IB9CW0-Text-Analytics/blob/main/day_seven_chunking_strategies.ipynb
> chunking required as we will retrieve the current trend in business or other field at the same time.

## Chunking

In [ ]:
!pip install llama_index.core
!pip install llama_index.readers.file

In [ ]:
# semantic chunking setp-up
!pip install llama_index_embeddings_huggingface

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the embedding model into LlamaIndex's settings
Settings.embed_model = embed_model


In [ ]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
import pandas as pd

# Initialize SemanticSplitterNodeParser
semantic_parser = SemanticSplitterNodeParser(buffer_size=1, breakpoint_percentile_threshold=90, embed_model=embed_model)

# Define the pipeline for semantic splitting
pipeline = [semantic_parser]

# Function to apply the pipeline to a text
def apply_pipeline(text, pipeline):
    doc = ww_df_c[description]
    for node_parser in pipeline:
        doc = node_parser.parse(doc)
    return doc

# Apply the pipeline to the 'description' column
ww_df_c['description_chunks'] = ww_df_c['description'].apply(lambda x: apply_pipeline(x, pipeline))

# Display the DataFrame with chunked descriptions
print(ww_df_c)


KeyError: 'MIAMI: The head of Saudi Arabia’s Public Investment Fund (PIF) on Thursday pitched the kingdom as a prospective hub for artificial intelligence activity outside the United States, citing its energy resources and funding capacity. “We are fairly well positioned to be an AI hub outside of the US,” said PIF Governor Yasir Al-Rumayyan, speaking at a Miami investment event sponsored by the sovereign wealth fund. “AI will consume a lot of energy and we are the global leader when it comes to fossil fuel energy and when it comes to renewable energy,” he said. Most Gulf markets fall in early trade, Saudi gains Rumayyan said Saudi Arabia, the world’s biggest oil producer, also had the “political will” to make AI projects happen and ample funds it could deploy to nurture the technology’s development. His comments signal that data centers may be a core part of Riyadh’s strategy to capitalize on booming demand for generative AI technology, which requires vast amounts of processing power.

In [ ]:
ww_df_c_dic

{'description': 0       MIAMI: The head of Saudi Arabia’s Public Inves...
 1       New Delhi: The Union government has amended th...
 2       The offshore wind power industry is navigating...
 3       They have signed a 25-year long-term power pur...
 4       The Government of India has approved amendment...
                               ...                        
 3746    The renewable energy company Drax welcomed 800...
 3747    article author: ReutersAuthor: Sat, 2023-12-23...
 3748    As the world pivots from planet-warming fossil...
 3751    OTTAWA: If Republican frontrunner Donald Trump...
 3752    As the world pivots from planet-warming fossil...
 Name: description, Length: 3199, dtype: object}

## Sentence Splitter

In [ ]:
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path # for finding the file

ram_docs = FlatReader().load_data(Path("/content/ram.txt"))

# we will limit to chunk size 100
parser = SentenceSplitter(chunk_size=100, chunk_overlap=0)
ram_nodes = parser.get_nodes_from_documents(ram_docs)

# 5. Build RAG

code reference: https://github.com/RDGopal/IB9CW0-Text-Analytics/blob/main/day_eight_rag_query_modes.ipynb

# 6. Connect LLM

# 7. Prompt engineering for LLM

# 8. Model evaluation